In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE  SCHEDULED_STOP
cluster-0016  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
from google.cloud import storage
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from graphframes import *
from pyspark.sql import Row
from pyspark.sql.functions import col, avg
from inverted_index_gcp import InvertedIndex
import math
from collections import defaultdict
import pickle

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  5 16:58 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'yoav-wiki-bucket-123'
full_path = f"gs://{bucket_name}/"
paths = []

client = storage.Client()
blobs = client.list_blobs(bucket_name)

for b in blobs:
    name = b.name

    if name == 'graphframes.sh':
        continue
    if name.startswith(('pr/', 'postings_gcp/', 'title_id_dict_parquet/')):
        continue
    if not name.endswith('.parquet'):
        continue

    paths.append(full_path + name)

***GCP setup is complete!***

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

In [ ]:
# Paths for parquets
TITLES_PATH  = f"gs://{bucket_name}/title_id_dict_parquet"
DOC_LEN_PATH = f"gs://{bucket_name}/doc_len_dict_parquet"

# Tokenizer setup
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)


def _doc_len(text: str) -> int:
    tokens = [m.group() for m in RE_WORD.finditer(text.lower())]
    tokens = [t for t in tokens if t not in all_stopwords]
    return len(tokens)

# Save titles parquet (id -> title)
(parquetFile
  .select(col("id").cast("long").alias("id"), col("title"))
  .write
  .mode("overwrite")
  .parquet(TITLES_PATH))

print("Titles parquet saved to:", TITLES_PATH)

# Save doc length parquet (id -> doc_len)
doc_len_rdd = doc_text_pairs.map(lambda x: Row(id=int(x[1]), doc_len=_doc_len(x[0])))
doc_len_df = spark.createDataFrame(doc_len_rdd)

(doc_len_df
  .write
  .mode("overwrite")
  .parquet(DOC_LEN_PATH))

print("Doc-len parquet saved to:", DOC_LEN_PATH)

Titles parquet saved to: gs://yoav-wiki-bucket-123/title_id_dict_parquet


In [ ]:
!gsutil ls gs://{bucket_name}/title_id_dict_parquet
!gsutil ls gs://{bucket_name}/doc_len_dict_parquet

gs://yoav-wiki-bucket-123/title_id_dict_parquet/
gs://yoav-wiki-bucket-123/title_id_dict_parquet/_SUCCESS
gs://yoav-wiki-bucket-123/title_id_dict_parquet/part-00000-5a188c2a-fdac-46f7-af52-7ad834324e41-c000.snappy.parquet
gs://yoav-wiki-bucket-123/title_id_dict_parquet/part-00002-5a188c2a-fdac-46f7-af52-7ad834324e41-c000.snappy.parquet
gs://yoav-wiki-bucket-123/title_id_dict_parquet/part-00005-5a188c2a-fdac-46f7-af52-7ad834324e41-c000.snappy.parquet
gs://yoav-wiki-bucket-123/title_id_dict_parquet/part-00007-5a188c2a-fdac-46f7-af52-7ad834324e41-c000.snappy.parquet
gs://yoav-wiki-bucket-123/title_id_dict_parquet/part-00009-5a188c2a-fdac-46f7-af52-7ad834324e41-c000.snappy.parquet
gs://yoav-wiki-bucket-123/title_id_dict_parquet/part-00010-5a188c2a-fdac-46f7-af52-7ad834324e41-c000.snappy.parquet
gs://yoav-wiki-bucket-123/title_id_dict_parquet/part-00012-5a188c2a-fdac-46f7-af52-7ad834324e41-c000.snappy.parquet
gs://yoav-wiki-bucket-123/title_id_dict_parquet/part-00013-5a188c2a-fdac-46f7-af52

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
NUM_BUCKETS = 124

def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

  filtered_words = []
  for word in tokens:
    if word not in all_stopwords:
      filtered_words.append(word)

  counts = Counter()
  for word in filtered_words:
    counts[word] += 1

  result = []
  for word, tf in counts.items():
    result.append((word, (id, tf)))

  return result

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key=lambda x: x[0])

def calculate_df(postings):
  return postings.mapValues(lambda pl: len(pl)) # compute df by counting how many doc_ids appear in its posting list

def partition_postings_and_write(postings):
  bucketed = postings.map(lambda x: (token2bucket_id(x[0]), (x[0], x[1]))) # (token, pl) -> (bucket_id, (token, pl))
  grouped = bucketed.groupByKey()  # group all (token, pl) pairs that belong to the same backet_id

  # For each bucket - write all its posting lists to disk and get back a dict {token:[(file_name, offset),...],...}
  result = grouped.map(
      lambda bucket_data:
                       InvertedIndex.write_a_posting_list(
                           (bucket_data[0], list(bucket_data[1])),
                           bucket_name))

  return result

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

In [ ]:
# test index construction time
assert index_const_time < 60*120

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.4 MiB/ 18.4 MiB]                                                
Operation completed over 1 objects/18.4 MiB.                                     


In [ ]:
!gsutil ls -lh $index_dst

 18.45 MiB  2026-01-05T19:06:56Z  gs://yoav-wiki-bucket-123/postings_gcp/index.pkl
TOTAL: 1 objects, 19341984 bytes (18.45 MiB)


In [ ]:
# Config
bucket_name = "yoav-wiki-bucket-123"
TITLES_PATH  = f"gs://{bucket_name}/title_id_dict_parquet"
DOC_LEN_PATH = f"gs://{bucket_name}/doc_len_dict_parquet"

# Load titles
titles_df = (
    spark.read.parquet(TITLES_PATH)
    .select(col("id").cast("long").alias("id"))
)

# Load doc lengths
doc_len_df = (
    spark.read.parquet(DOC_LEN_PATH)
    .select(col("id").cast("long").alias("id"),
            col("doc_len").cast("int").alias("doc_len"))
)

doc_len_dict = doc_len_df.rdd.map(lambda r: (r["id"], r["doc_len"])).collectAsMap()
avgdl = doc_len_df.agg(avg("doc_len")).first()[0]

print("titles_df loaded")
print("doc_len_dict size:", len(doc_len_dict))
print("avgdl:", avgdl)

Testing with 50 terms


Titles coverage for corpus sample: 50 / 50


In [ ]:
# Sanity check: titles coverage
sample_terms = list(inverted.df.keys())[:50]
corpus_ids = [int(r["id"]) for r in parquetFile.select("id").limit(50).collect()]
found = titles_df.where(col("id").isin(corpus_ids)).count()

In [ ]:
# Save TITLE_ID_DICT as pkl
PARQUET_PATH = "gs://yoav-wiki-bucket-123/title_id_dict_parquet"  # path to parquet folder in GCS
df = spark.read.parquet(PARQUET_PATH)   # read all parquet files in the folder
data = df.collect()

with open("title_id_dict.pkl", "wb") as f:
    pickle.dump(data, f)

BUCKET_DST = "gs://yoav-wiki-bucket-123/title_id_dict.pkl"
!gsutil cp title_id.pkl $BUCKET_DST

# PageRank

Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
def generate_graph(pages):
  # Edges - (src_id, dst_id) for every link in anchor_text
  edges = pages.flatMap(lambda row: ([] if row.anchor_text is None  # no link = no edge
                                     else [(row.id, dest.id) for dest in row.anchor_text if dest.id is not None])).distinct().cache()  # link = edge

  # Vertices - all unique page ids that appear as source or as destination
  vertex_ids = edges.flatMap(lambda e: [e[0], e[1]]).distinct()
  vertices = vertex_ids.map(lambda vid: (vid,)).cache() # convert id -> (id,)

  return edges, vertices

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://bucket_324041060/multistream*").select("id", "anchor_text").rdd
# construct the graph
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750| 9913.728782160775|
|  10568| 5385.349263642041|
|  32927| 5282.081575765278|
|  30680| 5128.233709604119|
|5843419| 4957.567686263868|
|  68253| 4769.278265355158|
|  31717|4486.3501805483065|
|  11867| 4146.414650912772|
|  14533|3996.4664408855037|
| 645042|3531.6270898037437|
|  17867|3246.0983906041415|
|5042916| 2991.945739166177|
|4689264| 2982.324883041747|
|  14532|2934.7468292031704|
|  25391| 2903.546223513398|
|   5405|2891.4163291546365|
|4764461|2834.3669873326603|
|  15573|2783.8651181588366|
|   9316|2782.0396464137702|
|8569916| 2775.286191840015|
+-------+------------------+
only showing top 20 rows



In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*120